# Alzheimer's Simulation Integration Tests

This notebook verifies the causal chain in the vivarium Alzheimer's simulation:

1. **Testing coverage** at key time points (2030, 2035) matches spec
2. **Testing -> Treatment**: positive BBBM tests drive treatment initiation
3. **Treatment -> Progression**: treatment slows BBBM -> MCI transition
4. **Baseline verification**: no BBBM testing or treatment in baseline scenario

### Spec reference (PR 1888)

| Parameter | Knots |
|---|---|
| BBBM testing rate | (2027, 0%) -> (2030.5, 10%) -> (2045, 50%) -> (2055, 60%) |
| Treatment probability | (2027, 0%) -> (2035.5, 30%) -> (2100, 80%) |
| Positive diagnosis probability | 50% |

### How to use

Run all cells. Assert statements at the end confirm key invariants.
If the notebook completes without error, all tests pass.

In [1]:
import math

import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from vivarium import InteractiveContext

SPEC_PATH = '../src/vivarium_csu_alzheimers/model_specifications/model_spec.yaml'
POPULATION_SIZE = 10_000
STEP_SIZE_DAYS = 182

BBBM_STATE = 'alzheimers_blood_based_biomarker_state'
MCI_STATE = 'alzheimers_mild_cognitive_impairment_state'
DEMENTIA_STATE = 'alzheimers_disease_state'
DISEASE_COL = 'alzheimers_disease_and_other_dementias'

## Helper functions

In [2]:
def step_to_year(sim, target_year, target_month=7):
    """Step the simulation forward with a progress bar."""
    target = pd.Timestamp(f'{target_year}-{target_month:02d}-01')
    est_steps = max(1, math.ceil((target - sim.current_time).days / STEP_SIZE_DAYS))
    steps = 0
    with tqdm(total=est_steps, desc=f'-> {target_year}', unit='step') as pbar:
        while sim.current_time < target:
            sim.step()
            steps += 1
            pbar.update(1)
        pbar.total = steps
        pbar.refresh()
    return sim

In [3]:
def get_eligible_for_bbbm(pop):
    """Return mask of simulants eligible for BBBM testing.

    Eligibility: BBBM state, age 65-80, no prior positive test, alive.
    (Ignores retesting interval -- this is the 'broadly eligible' population.)
    """
    return (
        (pop[DISEASE_COL] == BBBM_STATE)
        & (pop['age'] >= 65)
        & (pop['age'] < 80)
        & (pop['bbbm_test_result'] != 'positive')
        & (pop['alive'] == 'alive')
    )

# Demo
_demo = pd.DataFrame({
    DISEASE_COL: ['susceptible', BBBM_STATE, BBBM_STATE, BBBM_STATE],
    'age': [66, 79, 64, 70],
    'bbbm_test_result': ['not_tested', 'not_tested', 'not_tested', 'positive'],
    'alive': ['alive', 'alive', 'alive', 'alive'],
})
print(get_eligible_for_bbbm(_demo).tolist(), ' (expect [False, True, False, False])')

[False, True, False, False]  (expect [False, True, False, False])


In [ ]:
def testing_summary(pop, label=''):
    """Print and return a summary of BBBM testing state."""
    eligible = get_eligible_for_bbbm(pop)
    n_eligible = int(eligible.sum())
    n_alive = int((pop['alive'] == 'alive').sum())
    tested_ever = eligible & pop['bbbm_test_date'].notna()
    n_tested = int(tested_ever.sum())
    results = pop.loc[pop['alive'] == 'alive', 'bbbm_test_result'].value_counts()

    print(f'\n=== {label} ===')
    print(f'Alive: {n_alive}  |  Eligible for BBBM test: {n_eligible}')
    if n_eligible > 0:
        print(f'Eligible & ever tested: {n_tested} ({n_tested/n_eligible*100:.1f}%)')
    print(f'BBBM test results (all alive):\n{results.to_string()}')
    return {'n_eligible': n_eligible, 'n_tested': n_tested, 'n_alive': n_alive}

testing_summary(_demo, 'demonstration of testing_summary')

In [ ]:
def treatment_summary(pop, label=''):
    """Print and return a summary of treatment state."""
    alive = pop['alive'] == 'alive'
    states = pop.loc[alive, 'treatment'].value_counts()
    positive = alive & (pop['bbbm_test_result'] == 'positive')
    n_positive = int(positive.sum())

    active = ['waiting_for_treatment', 'treatment_effect',
              'waning_effect', 'no_effect_after_treatment']
    n_active = int((positive & pop['treatment'].isin(active)).sum())
    n_declined = int((positive & (pop['treatment'] == 'no_effect_never_treated')).sum())

    print(f'\n=== {label} ===')
    print(f'Treatment states (alive):\n{states.to_string()}')
    print(f'\nBBBM-positive: {n_positive}')
    if n_positive > 0:
        print(f'  In treatment pipeline: {n_active} ({n_active/n_positive*100:.1f}%)')
        print(f'  Declined treatment:    {n_declined} ({n_declined/n_positive*100:.1f}%)')
    return {'n_positive': n_positive, 'n_active': n_active, 'n_declined': n_declined}

treatment_summary(_demo, 'demonstration of treatment_summary')

In [ ]:
def disease_summary(pop, label=''):
    """Print and return a dict of disease state counts."""
    alive = pop['alive'] == 'alive'
    states = pop.loc[alive, DISEASE_COL].value_counts()
    print(f'\n=== {label} ===')
    print(f'Disease states (alive):\n{states.to_string()}')
    return {k: int(v) for k, v in states.items()}

disease_summary(_demo, 'demonstration of disease_summary')

---
## Test 1: BBBM Testing Coverage at 2030 and 2035

The BBBM testing rate is a propensity threshold. At mid-2030 the rate is 10%, so only
simulants with `testing_propensity < 0.10` should be tested. At mid-2035, the threshold
is ~23.8%.

**Key check**: simulants *above* the threshold should almost never be tested (0-1%).
Simulants *below* the threshold may not all show as tested in a snapshot because new
simulants continually enter via incidence and are assigned a future first-test date
(by design, to avoid unrealistic testing surges).

In [7]:
sim = InteractiveContext(
    SPEC_PATH,
    configuration={
        'population': {'population_size': POPULATION_SIZE},
        'intervention': {'scenario': 'bbbm_testing_and_treatment'},
    }
)
print(f'Initialized at {sim.current_time}, pop={len(sim.get_population())}')
testing_summary(sim.get_population(), 'Initial state (2022)')

2026-02-07 11:17:33.248 | INFO     | simulation_1-artifact_manager:79 - Running simulation from artifact located at /home/abie/vivarium_csu_alzheimers/united_states_of_america.hdf.
2026-02-07 11:17:33.249 | INFO     | simulation_1-artifact_manager:80 - Artifact base filter terms are ['draw == 0'].
2026-02-07 11:17:33.249 | INFO     | simulation_1-artifact_manager:81 - Artifact additional filter terms are None.


/home/abie/miniforge3/envs/vivarium_csu_alzheimers_simulation/lib/python3.11/site-packages/vivarium_public_health/plugins/parser.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


2026-02-07 11:17:34.680 | INFO     | simulation_1-results_context:129 - The following stratifications are registered but not used by any observers: 
['current_year', 'semester']
2026-02-07 11:17:34.787 | WARNING  | simulation_1-resource_manager:176 - Resource stream.alzheimers_disease_and_other_dementias_initial_states is not produced by any component but is needed to compute (column.alzheimers_disease_and_other_dementias).
2026-02-07 11:17:34.788 | WARNING  | simulation_1-resource_manager:176 - Resource column.treatment_effect.dwell_time is not produced by any component but is needed to compute (value_modifier.treatment_effect.dwell_time.1.treatment.modify_dwell_time).
2026-02-07 11:17:34.789 | WARNING  | simulation_1-resource_manager:176 - Resource column.waning_effect.dwell_time is not produced by any component but is needed to compute (value_modifier.waning_effect.dwell_time.1.treatment.modify_dwell_time).
2026-02-07 11:17:34.790 | WARNING  | simulation_1-resource_manager:176 - Res

{'n_eligible': 2192, 'n_tested': 0, 'n_alive': 10000}

In [8]:
step_to_year(sim, 2030, target_month=7)
pop_2030 = sim.get_population()
stats_test_2030 = testing_summary(pop_2030, f'Testing at {sim.current_time.date()}')

RATE_2030 = 0.10
elig = pop_2030[get_eligible_for_bbbm(pop_2030)]
low = elig[elig['testing_propensity'] < RATE_2030]
high = elig[elig['testing_propensity'] >= RATE_2030]
low_tested_2030 = int(low['bbbm_test_date'].notna().sum())
high_tested_2030 = int(high['bbbm_test_date'].notna().sum())

print(f'\nPropensity check (threshold {RATE_2030:.0%}):')
print(f'  Below: {len(low)} eligible, {low_tested_2030} tested ({low_tested_2030/max(len(low),1)*100:.1f}%)')
print(f'  Above: {len(high)} eligible, {high_tested_2030} tested ({high_tested_2030/max(len(high),1)*100:.1f}%)')

high_tested_frac_2030 = high_tested_2030 / max(len(high), 1)

-> 2030:   0%|          | 0/18 [00:00<?, ?step/s]

2026-02-07 11:17:36.103 | INFO     | simulation_1 - vivarium.framework.engine:284 - 2022-01-01 00:00:00
2026-02-07 11:17:39.329 | INFO     | simulation_1 - vivarium.framework.engine:284 - 2022-07-02 00:00:00
2026-02-07 11:17:42.299 | INFO     | simulation_1 - vivarium.framework.engine:284 - 2022-12-31 00:00:00
2026-02-07 11:17:46.341 | INFO     | simulation_1 - vivarium.framework.engine:284 - 2023-07-01 00:00:00
2026-02-07 11:17:50.905 | INFO     | simulation_1 - vivarium.framework.engine:284 - 2023-12-30 00:00:00
2026-02-07 11:17:56.203 | INFO     | simulation_1 - vivarium.framework.engine:284 - 2024-06-29 00:00:00
2026-02-07 11:18:06.800 | INFO     | simulation_1 - vivarium.framework.engine:284 - 2024-12-28 00:00:00
2026-02-07 11:18:12.514 | INFO     | simulation_1 - vivarium.framework.engine:284 - 2025-06-28 00:00:00
2026-02-07 11:18:17.788 | INFO     | simulation_1 - vivarium.framework.engine:284 - 2025-12-27 00:00:00
2026-02-07 11:18:22.546 | INFO     | simulation_1 - vivarium.fra

In [9]:
step_to_year(sim, 2035, target_month=7)
pop_2035 = sim.get_population()
stats_test_2035 = testing_summary(pop_2035, f'Testing at {sim.current_time.date()}')

# Interpolated between (2030.5, 10%) and (2045.0, 50%)
RATE_2035 = 0.10 + 0.40 * (5.0 / 14.5)  # ~23.8%
elig = pop_2035[get_eligible_for_bbbm(pop_2035)]
low = elig[elig['testing_propensity'] < RATE_2035]
high = elig[elig['testing_propensity'] >= RATE_2035]
low_tested_2035 = int(low['bbbm_test_date'].notna().sum())
high_tested_2035 = int(high['bbbm_test_date'].notna().sum())

print(f'\nPropensity check (threshold {RATE_2035:.1%}):')
print(f'  Below: {len(low)} eligible, {low_tested_2035} tested ({low_tested_2035/max(len(low),1)*100:.1f}%)')
print(f'  Above: {len(high)} eligible, {high_tested_2035} tested ({high_tested_2035/max(len(high),1)*100:.1f}%)')

low_tested_frac_2035 = low_tested_2035 / max(len(low), 1)
high_tested_frac_2035 = high_tested_2035 / max(len(high), 1)

-> 2035:   0%|          | 0/10 [00:00<?, ?step/s]

2026-02-07 11:19:24.990 | INFO     | simulation_1 - vivarium.framework.engine:284 - 2030-12-21 00:00:00
2026-02-07 11:19:33.770 | INFO     | simulation_1 - vivarium.framework.engine:284 - 2031-06-21 00:00:00
2026-02-07 11:19:40.928 | INFO     | simulation_1 - vivarium.framework.engine:284 - 2031-12-20 00:00:00
2026-02-07 11:19:53.724 | INFO     | simulation_1 - vivarium.framework.engine:284 - 2032-06-19 00:00:00
2026-02-07 11:20:03.290 | INFO     | simulation_1 - vivarium.framework.engine:284 - 2032-12-18 00:00:00
2026-02-07 11:20:12.857 | INFO     | simulation_1 - vivarium.framework.engine:284 - 2033-06-18 00:00:00
2026-02-07 11:20:25.745 | INFO     | simulation_1 - vivarium.framework.engine:284 - 2033-12-17 00:00:00
2026-02-07 11:20:35.529 | INFO     | simulation_1 - vivarium.framework.engine:284 - 2034-06-17 00:00:00
2026-02-07 11:20:45.306 | INFO     | simulation_1 - vivarium.framework.engine:284 - 2034-12-16 00:00:00
2026-02-07 11:20:56.368 | INFO     | simulation_1 - vivarium.fra

### Why are only ~50% of below-threshold simulants tested?

New simulants enter via `AlzheimersIncidence` each step. On entry, they're assigned a
future `next_bbbm_test_date` (0-4.5 years out) to avoid testing surges. At any snapshot,
recent entrants haven't had their first test yet. Let's verify this is the explanation:

In [10]:
elig = pop_2035[get_eligible_for_bbbm(pop_2035)]
low = elig[elig['testing_propensity'] < RATE_2035]
untested = low[low['bbbm_test_date'].isna()]
tested = low[low['bbbm_test_date'].notna()]

n_untested_future_date = int((untested['next_bbbm_test_date'] > sim.current_time).sum())
n_untested_nat = int(untested['next_bbbm_test_date'].isna().sum())
n_untested_after_2030 = int((untested['entrance_time'] > pd.Timestamp('2030-01-01')).sum())
n_untested_at_start = int((untested['entrance_time'] < pd.Timestamp('2023-01-01')).sum())
all_untested_are_recent = bool((untested['entrance_time'] > pd.Timestamp('2030-01-01')).all())

print(f'Untested below-threshold eligible: {len(untested)}')
print(f'  next_bbbm_test_date in future: {n_untested_future_date}')
print(f'  next_bbbm_test_date is NaT:    {n_untested_nat}')
print(f'  Entered sim after 2030:        {n_untested_after_2030} of {len(untested)}')
print(f'  Entered sim at start (2022):   {n_untested_at_start}')
print(f'\nAll untested are recent entrants (post-2030): {all_untested_are_recent}')
print('-> This confirms the ~50% rate is from new entrants awaiting their first test, not a bug.')

Untested below-threshold eligible: 338
  next_bbbm_test_date in future: 323
  next_bbbm_test_date is NaT:    15
  Entered sim after 2030:        338 of 338
  Entered sim at start (2022):   0

All untested are recent entrants (post-2030): True
-> This confirms the ~50% rate is from new entrants awaiting their first test, not a bug.


---
## Test 2: Testing Drives Treatment

At mid-2035, treatment probability is ~30%. Only BBBM-positive simulants should enter
the treatment pipeline. Simulants with `treatment_propensity >= 0.30` should not be treated.

In [11]:
stats_treat_2035 = treatment_summary(pop_2035, f'Treatment at {sim.current_time.date()}')

# Verify: no treatment without positive test
alive = pop_2035['alive'] == 'alive'
not_positive = pop_2035['bbbm_test_result'] != 'positive'
non_susceptible_states = ['waiting_for_treatment', 'treatment_effect',
                          'waning_effect', 'no_effect_after_treatment', 'no_effect_never_treated']
wrongly_treated = alive & not_positive & pop_2035['treatment'].isin(non_susceptible_states)
n_wrongly_treated_2035 = int(wrongly_treated.sum())
print(f'\nSimulants in treatment WITHOUT positive test: {n_wrongly_treated_2035} (should be 0)')


=== Treatment at 2035-12-15 ===
Treatment states (alive):
treatment
susceptible_to_treatment     17360
no_effect_never_treated        606
treatment_effect               118
waiting_for_treatment           27
waning_effect                    8
no_effect_after_treatment        1

BBBM-positive: 760
  In treatment pipeline: 154 (20.3%)
  Declined treatment:    606 (79.7%)

Simulants in treatment WITHOUT positive test: 0 (should be 0)


In [12]:
# Treatment propensity check
TREATMENT_RATE_2035 = 0.30
positive = pop_2035[(pop_2035['alive'] == 'alive') & (pop_2035['bbbm_test_result'] == 'positive')]
active_states = ['waiting_for_treatment', 'treatment_effect',
                  'waning_effect', 'no_effect_after_treatment']

low = positive[positive['treatment_propensity'] < TREATMENT_RATE_2035]
high = positive[positive['treatment_propensity'] >= TREATMENT_RATE_2035]
low_treated = int(low['treatment'].isin(active_states).sum())
high_treated = int(high['treatment'].isin(active_states).sum())

print(f'Treatment propensity check (threshold {TREATMENT_RATE_2035:.0%}):')
print(f'  Below: {len(low)} positive, {low_treated} in treatment ({low_treated/max(len(low),1)*100:.1f}%)')
print(f'  Above: {len(high)} positive, {high_treated} in treatment ({high_treated/max(len(high),1)*100:.1f}%)')

high_treated_frac_2035 = high_treated / max(len(high), 1)

Treatment propensity check (threshold 30%):
  Below: 224 positive, 154 in treatment (68.8%)
  Above: 536 positive, 0 in treatment (0.0%)


---
## Test 3: Treatment Reduces BBBM -> MCI Progression

Treatment applies a relative risk (RR ~0.4-0.6) to the BBBM->MCI transition rate.
We compare `bbbm_testing_and_treatment` vs `bbbm_testing` at 2045, where testing and
treatment rates are high enough for a visible signal.

In [13]:
step_to_year(sim, 2045, target_month=1)
pop_treat_2045 = sim.get_population()
disease_treat = disease_summary(pop_treat_2045, f'Treatment scenario at {sim.current_time.date()}')
treatment_summary(pop_treat_2045, f'Treatment states at {sim.current_time.date()}')

-> 2045:   0%|          | 0/19 [00:00<?, ?step/s]

2026-02-07 11:21:07.160 | INFO     | simulation_1 - vivarium.framework.engine:284 - 2035-12-15 00:00:00
2026-02-07 11:21:16.904 | INFO     | simulation_1 - vivarium.framework.engine:284 - 2036-06-14 00:00:00
2026-02-07 11:21:25.896 | INFO     | simulation_1 - vivarium.framework.engine:284 - 2036-12-13 00:00:00
2026-02-07 11:21:37.376 | INFO     | simulation_1 - vivarium.framework.engine:284 - 2037-06-13 00:00:00
2026-02-07 11:21:46.722 | INFO     | simulation_1 - vivarium.framework.engine:284 - 2037-12-12 00:00:00
2026-02-07 11:21:55.472 | INFO     | simulation_1 - vivarium.framework.engine:284 - 2038-06-12 00:00:00
2026-02-07 11:22:08.911 | INFO     | simulation_1 - vivarium.framework.engine:284 - 2038-12-11 00:00:00
2026-02-07 11:22:18.617 | INFO     | simulation_1 - vivarium.framework.engine:284 - 2039-06-11 00:00:00
2026-02-07 11:22:27.761 | INFO     | simulation_1 - vivarium.framework.engine:284 - 2039-12-10 00:00:00
2026-02-07 11:22:40.666 | INFO     | simulation_1 - vivarium.fra

{'n_positive': 2023, 'n_active': 674, 'n_declined': 1349}

In [14]:
print('Setting up testing-only scenario for comparison...')
sim_testing = InteractiveContext(
    SPEC_PATH,
    configuration={
        'population': {'population_size': POPULATION_SIZE},
        'intervention': {'scenario': 'bbbm_testing'},
    }
)
step_to_year(sim_testing, 2045, target_month=1)
pop_test_2045 = sim_testing.get_population()
disease_test = disease_summary(pop_test_2045, f'Testing-only at {sim_testing.current_time.date()}')

Setting up testing-only scenario for comparison...
2026-02-07 11:24:26.956 | INFO     | simulation_2-artifact_manager:79 - Running simulation from artifact located at /home/abie/vivarium_csu_alzheimers/united_states_of_america.hdf.
2026-02-07 11:24:26.958 | INFO     | simulation_2-artifact_manager:80 - Artifact base filter terms are ['draw == 0'].
2026-02-07 11:24:26.958 | INFO     | simulation_2-artifact_manager:81 - Artifact additional filter terms are None.
2026-02-07 11:24:29.104 | INFO     | simulation_2-results_context:129 - The following stratifications are registered but not used by any observers: 
['current_year', 'semester']
2026-02-07 11:24:29.280 | WARNING  | simulation_2-resource_manager:176 - Resource stream.alzheimers_disease_and_other_dementias_initial_states is not produced by any component but is needed to compute (column.alzheimers_disease_and_other_dementias).
2026-02-07 11:24:29.282 | WARNING  | simulation_2-resource_manager:176 - Resource column.treatment_effect.d

-> 2045:   0%|          | 0/47 [00:00<?, ?step/s]

2026-02-07 11:24:31.082 | INFO     | simulation_2 - vivarium.framework.engine:284 - 2022-01-01 00:00:00
2026-02-07 11:24:36.368 | INFO     | simulation_2 - vivarium.framework.engine:284 - 2022-07-02 00:00:00
2026-02-07 11:24:41.097 | INFO     | simulation_2 - vivarium.framework.engine:284 - 2022-12-31 00:00:00
2026-02-07 11:24:45.099 | INFO     | simulation_2 - vivarium.framework.engine:284 - 2023-07-01 00:00:00
2026-02-07 11:24:49.812 | INFO     | simulation_2 - vivarium.framework.engine:284 - 2023-12-30 00:00:00
2026-02-07 11:24:54.224 | INFO     | simulation_2 - vivarium.framework.engine:284 - 2024-06-29 00:00:00
2026-02-07 11:25:01.092 | INFO     | simulation_2 - vivarium.framework.engine:284 - 2024-12-28 00:00:00
2026-02-07 11:25:05.226 | INFO     | simulation_2 - vivarium.framework.engine:284 - 2025-06-28 00:00:00
2026-02-07 11:25:09.839 | INFO     | simulation_2 - vivarium.framework.engine:284 - 2025-12-27 00:00:00
2026-02-07 11:25:15.521 | INFO     | simulation_2 - vivarium.fra

In [15]:
print('\n=== Scenario Comparison at 2045 ===')
print(f'{"State":<25} {"Testing Only":>14} {"Test+Treat":>11} {"Diff":>8}')
print('-' * 60)
for state in [BBBM_STATE, MCI_STATE, DEMENTIA_STATE]:
    n_t = disease_test.get(state, 0)
    n_tt = disease_treat.get(state, 0)
    label = state.replace('alzheimers_', '').replace('_state', '').replace('_', ' ')
    print(f'{label:<25} {n_t:>14} {n_tt:>11} {n_tt - n_t:>+8}')

mci_test = disease_test.get(MCI_STATE, 0)
mci_treat = disease_treat.get(MCI_STATE, 0)
bbbm_test = disease_test.get(BBBM_STATE, 0)
bbbm_treat = disease_treat.get(BBBM_STATE, 0)

print(f'\nTreatment reduces MCI count: {mci_treat < mci_test}')
print(f'Treatment retains more in BBBM: {bbbm_treat > bbbm_test}')


=== Scenario Comparison at 2045 ===
State                       Testing Only  Test+Treat     Diff
------------------------------------------------------------
blood based biomarker               9529        9625      +96
mild cognitive impairment           4405        4359      -46
disease                             6403        6363      -40

Treatment reduces MCI count: True
Treatment retains more in BBBM: True


---
## Test 4: Baseline Verification

In the baseline scenario (no BBBM testing, no treatment):
- No BBBM tests should occur
- All simulants should remain in `susceptible_to_treatment`
- CSF/PET baseline testing should still work for MCI/Dementia simulants

In [16]:
print('Setting up baseline scenario...')
sim_baseline = InteractiveContext(
    SPEC_PATH,
    configuration={
        'population': {'population_size': POPULATION_SIZE},
        'intervention': {'scenario': 'baseline'},
    }
)
step_to_year(sim_baseline, 2035, target_month=7)
pop_baseline = sim_baseline.get_population()
alive_bl = pop_baseline['alive'] == 'alive'

n_bbbm_tested_baseline = int((pop_baseline.loc[alive_bl, 'bbbm_test_result'] != 'not_tested').sum())
n_non_susceptible_baseline = int((pop_baseline.loc[alive_bl, 'treatment'] != 'susceptible_to_treatment').sum())
csf_pet = pop_baseline.loc[alive_bl, 'testing_state'].value_counts()

print(f'\nBaseline at {sim_baseline.current_time.date()}:')
print(f'BBBM tests: {n_bbbm_tested_baseline} (should be 0)')
print(f'Non-susceptible treatment: {n_non_susceptible_baseline} (should be 0)')
print(f'\nCSF/PET testing (should have nonzero CSF and PET):\n{csf_pet.to_string()}')

Setting up baseline scenario...
2026-02-07 11:30:34.437 | INFO     | simulation_3-artifact_manager:79 - Running simulation from artifact located at /home/abie/vivarium_csu_alzheimers/united_states_of_america.hdf.
2026-02-07 11:30:34.439 | INFO     | simulation_3-artifact_manager:80 - Artifact base filter terms are ['draw == 0'].
2026-02-07 11:30:34.441 | INFO     | simulation_3-artifact_manager:81 - Artifact additional filter terms are None.
2026-02-07 11:30:36.822 | INFO     | simulation_3-results_context:129 - The following stratifications are registered but not used by any observers: 
['current_year', 'semester']
2026-02-07 11:30:36.955 | WARNING  | simulation_3-resource_manager:176 - Resource stream.alzheimers_disease_and_other_dementias_initial_states is not produced by any component but is needed to compute (column.alzheimers_disease_and_other_dementias).
2026-02-07 11:30:36.956 | WARNING  | simulation_3-resource_manager:176 - Resource column.treatment_effect.dwell_time is not pr

-> 2035:   0%|          | 0/28 [00:00<?, ?step/s]

2026-02-07 11:30:38.675 | INFO     | simulation_3 - vivarium.framework.engine:284 - 2022-01-01 00:00:00
2026-02-07 11:30:43.246 | INFO     | simulation_3 - vivarium.framework.engine:284 - 2022-07-02 00:00:00
2026-02-07 11:30:51.162 | INFO     | simulation_3 - vivarium.framework.engine:284 - 2022-12-31 00:00:00
2026-02-07 11:30:57.869 | INFO     | simulation_3 - vivarium.framework.engine:284 - 2023-07-01 00:00:00
2026-02-07 11:31:02.497 | INFO     | simulation_3 - vivarium.framework.engine:284 - 2023-12-30 00:00:00
2026-02-07 11:31:08.031 | INFO     | simulation_3 - vivarium.framework.engine:284 - 2024-06-29 00:00:00
2026-02-07 11:31:12.786 | INFO     | simulation_3 - vivarium.framework.engine:284 - 2024-12-28 00:00:00
2026-02-07 11:31:17.853 | INFO     | simulation_3 - vivarium.framework.engine:284 - 2025-06-28 00:00:00
2026-02-07 11:31:25.654 | INFO     | simulation_3 - vivarium.framework.engine:284 - 2025-12-27 00:00:00
2026-02-07 11:31:30.750 | INFO     | simulation_3 - vivarium.fra

---
## Assertions

Automated checks confirming the key invariants. If all cells above ran correctly,
these should pass. A successful run-all-cells = all tests pass.

In [17]:
# Test 1: Propensity threshold correctly filters testing
assert high_tested_frac_2030 < 0.05, (
    f'Above-threshold tested at 2030: {high_tested_frac_2030:.1%} (expected <5%)')
assert high_tested_frac_2035 < 0.05, (
    f'Above-threshold tested at 2035: {high_tested_frac_2035:.1%} (expected <5%)')
assert low_tested_frac_2035 > 0.20, (
    f'Below-threshold tested at 2035: {low_tested_frac_2035:.1%} (expected >20%)')
print('PASS: Test 1 - Propensity threshold correctly gates BBBM testing')

# Test 1b: Untested below-threshold simulants are recent entrants, not a bug
assert all_untested_are_recent, (
    'Some untested below-threshold simulants entered before 2030 -- possible bug')
print('PASS: Test 1b - All untested below-threshold simulants are recent entrants')

PASS: Test 1 - Propensity threshold correctly gates BBBM testing
PASS: Test 1b - All untested below-threshold simulants are recent entrants


In [18]:
# Test 2: Treatment only occurs with positive BBBM test
assert n_wrongly_treated_2035 == 0, (
    f'{n_wrongly_treated_2035} simulants in treatment without positive BBBM test')
print('PASS: Test 2a - No treatment without positive BBBM test')

assert high_treated_frac_2035 == 0.0, (
    f'Above-threshold treatment: {high_treated_frac_2035:.1%} (expected 0%)')
print('PASS: Test 2b - Treatment propensity threshold correctly gates treatment')

assert stats_treat_2035['n_active'] > 0, 'No simulants in treatment at 2035'
print(f'PASS: Test 2c - {stats_treat_2035["n_active"]} simulants actively in treatment at 2035')

PASS: Test 2a - No treatment without positive BBBM test
PASS: Test 2b - Treatment propensity threshold correctly gates treatment
PASS: Test 2c - 154 simulants actively in treatment at 2035


In [19]:
# Test 3: Treatment reduces MCI progression
assert mci_treat < mci_test, (
    f'Treatment MCI ({mci_treat}) >= testing-only MCI ({mci_test})')
print(f'PASS: Test 3a - Treatment reduces MCI ({mci_treat} < {mci_test}, diff={mci_treat - mci_test})')

assert bbbm_treat > bbbm_test, (
    f'Treatment BBBM ({bbbm_treat}) <= testing-only BBBM ({bbbm_test})')
print(f'PASS: Test 3b - Treatment retains more in BBBM ({bbbm_treat} > {bbbm_test}, diff=+{bbbm_treat - bbbm_test})')

PASS: Test 3a - Treatment reduces MCI (4359 < 4405, diff=-46)
PASS: Test 3b - Treatment retains more in BBBM (9625 > 9529, diff=+96)


In [20]:
# Test 4: Baseline has no BBBM testing or treatment
assert n_bbbm_tested_baseline == 0, f'{n_bbbm_tested_baseline} BBBM tests in baseline'
print('PASS: Test 4a - No BBBM tests in baseline scenario')

assert n_non_susceptible_baseline == 0, f'{n_non_susceptible_baseline} non-susceptible in baseline'
print('PASS: Test 4b - No treatment activity in baseline scenario')

assert csf_pet.get('csf', 0) > 0 and csf_pet.get('pet', 0) > 0, 'No CSF/PET testing in baseline'
print('PASS: Test 4c - CSF/PET baseline testing still works')

print('\n=== ALL TESTS PASSED ===')

PASS: Test 4a - No BBBM tests in baseline scenario
PASS: Test 4b - No treatment activity in baseline scenario
PASS: Test 4c - CSF/PET baseline testing still works

=== ALL TESTS PASSED ===
